In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../data/black_box_combined/seg_train/'
test_path = '../data/black_box_combined/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                   target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 28068 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 6000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-30 13:38:28.717598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-30 13:38:28.717722: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/10


2022-08-30 13:38:29.175950: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-30 13:38:30.169845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - ETA: 0s - loss: 1.3613 - accuracy: 0.4681

2022-08-30 13:39:38.644685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


878/878 [==============================] - 76s 84ms/step - loss: 1.3613 - accuracy: 0.4681 - val_loss: 1.2043 - val_accuracy: 0.5175
Epoch 2/10
878/878 [==============================] - 69s 79ms/step - loss: 1.0275 - accuracy: 0.6012 - val_loss: 1.1775 - val_accuracy: 0.5335
Epoch 3/10
878/878 [==============================] - 62s 70ms/step - loss: 0.9095 - accuracy: 0.6609 - val_loss: 1.5233 - val_accuracy: 0.4448
Epoch 4/10
878/878 [==============================] - 67s 76ms/step - loss: 0.8645 - accuracy: 0.6812 - val_loss: 0.8024 - val_accuracy: 0.6968
Epoch 5/10
878/878 [==============================] - 68s 77ms/step - loss: 0.7873 - accuracy: 0.7173 - val_loss: 0.8950 - val_accuracy: 0.6515
Epoch 6/10
878/878 [==============================] - 62s 70ms/step - loss: 0.7486 - accuracy: 0.7300 - val_loss: 1.2826 - val_accuracy: 0.5122
Epoch 7/10
878/878 [==============================] - 60s 68ms/step - loss: 0.7080 - accuracy: 0.7465 - val_loss: 1.3795 - val_accuracy: 0.5127
Epo

In [9]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

188/188 - 4s - loss: 0.7699 - accuracy: 0.7115 - 4s/epoch - 22ms/step
Accuracy of the model on the test set:  0.7114999890327454
